In [1]:
# import packages

import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

import math

# first, we need to import our essentia module.
import essentia

# as there are 2 operating modes in essentia which have the same algorithms,
# these latter are dispatched into 2 submodules:
import essentia.standard
import essentia.streaming

# # pylab contains the plot() function, as well as figure, etc... (same names as Matlab)
# from pylab import plot, show, figure, imshow
# %matplotlib inline
# import matplotlib.pyplot as plt

from essentia.standard import *


# import machine learning packages
import theano

from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

from sklearn.metrics import confusion_matrix

import tensorflow as tf

import dill

Using TensorFlow backend.


In [2]:
train = pd.read_pickle("jake_data/train_all.pkl")
test = pd.read_pickle("jake_data/test_all.pkl")

In [3]:
# pull out some data to make it better distributed
train = train[0:21600]

In [4]:
# shuffle dataframe
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

In [5]:
# need to figure out solution for different audio lenghts, jagged arrays?

X_train = []
Y_train = []
X_test = []
Y_test = []

X_test = np.zeros(shape=(len(test),1,13,100))
c = 0
for i in test["mfccs_files"]:
    l = i.shape[1]
    b = int(math.floor((l-100)/2))
    tp = i[0:13,b:(b + 100)]
    X_test[c,0,:,:] = tp
    c = c + 1
    
X_train = np.zeros(shape=(len(train),1,13,100))
c = 0
for i in train["mfccs_files"]:
    l = i.shape[1]
    b = int(math.floor((l-100)/2))
    tp = i[0:13,b:(b + 100)]
    X_train[c,0,:,:] = tp
    c = c + 1

Y_train = np.zeros(shape=(len(train),2)) 

c = 0
b = 0
s = 0
for i in train["output"]:
    if (i == "bonafide"):
        b = b + 1
        Y_train[c,:] = [np.float64(0.0),np.float64(1.0)]
    else:
        s = s + 1
        Y_train[c,:] = [np.float64(1.0),np.float64(0.0)]
    c = c + 1
print(b)
print(s)

Y_test = np.zeros(shape=(len(test),2)) 

c = 0
b = 0
s = 0
for i in test["output"]:
    if (i == "bonafide"):
        b = b + 1
        Y_test[c,:] = [np.float64(0.0),np.float64(1.0)]
    else:
        s = s + 1
        Y_test[c,:] = [np.float64(1.0),np.float64(0.0)]
    c = c + 1
       
print(b)
print(s)

5400
16200
5400
24300


In [6]:
X_train = X_train.reshape(X_train.shape[0], 1, 13,100)
X_test = X_test.reshape(X_test.shape[0], 1, 13,100)

In [7]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(1,13,100),data_format='channels_first'))

model.add(Convolution2D(32, 3, 3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(ELU(alpha=1.0)) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(32, 3, 3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(ELU(alpha=1.0)) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

W0826 14:43:14.252031 4727473600 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 14:43:14.286952 4727473600 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 14:43:14.298325 4727473600 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 14:43:14.314349 4727473600 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0826 14:43:14.315926 4727473600 deprecation_wrapper

In [8]:
import dill
from sklearn.utils import class_weight

# class_weights = {0: .99, 1: .01}

# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_train),
#                                                  y_train)

# class_weights = dict(enumerate(class_weights))
# print(class_weights)
# print(type(class_weights))

# compile to experement with better results, try not to get stuck at local min
# model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer='adam')

# normal compile
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0826 14:43:14.897367 4727473600 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [10]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

model.fit(X_train, Y_train,
               epochs=20,
               shuffle=True,
               batch_size=300,
#                class_weight=class_weights,
               verbose=1)

# model.fit(X_train, Y_train, 
#           batch_size=300, epochs=10, verbose=1)

W0826 14:43:24.131069 4727473600 deprecation.py:323] From /usr/local/lib/python2.7/site-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
21600/21600 [==============================] - 34s 2ms/step - loss: 0.6239 - acc: 0.7156
Epoch 2/20
21600/21600 [==============================] - 34s 2ms/step - loss: 0.5140 - acc: 0.7495
Epoch 3/20
21600/21600 [==============================] - 33s 2ms/step - loss: 0.4858 - acc: 0.7498
Epoch 4/20
21600/21600 [==============================] - 33s 2ms/step - loss: 0.4611 - acc: 0.7500
Epoch 5/20
21600/21600 [==============================] - 33s 2ms/step - loss: 0.4372 - acc: 0.7500
Epoch 6/20
21600/21600 [==============================] - 33s 2ms/step - loss: 0.4167 - acc: 0.7500
Epoch 7/20
21600/21600 [==============================] - 33s 2ms/step - loss: 0.3975 - acc: 0.7500
Epoch 8/20
21600/21600 [==============================] - 33s 2ms/step - loss: 0.3878 - acc: 0.7708
Epoch 9/20
21600/21600 [==============================] - 34s 2ms/step - loss: 0.3772 - acc: 0.8028
Epoch 10/20
21600/21600 [==============================] - 33s 2ms/step - loss: 0.3692 - acc: 0.8108

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("testing accuracy " + str(score))

score = model.evaluate(X_train, Y_train, verbose=0)
print("training accuracy " + str(score))

In [ ]:
y_train = []
s = 0
b = 0
for i in Y_train:
    if (i[0] == 0.0):
        b = b + 1
        y_train.append(0)
    else:
        s = s + 1
        y_train.append(1)
 
y_train = np.array(y_train)
print("training set: ")
print("0 values " + str(b))
print("1 values " + str(s))

y_test = []
s = 0
b = 0

for i in Y_test:
    if (i[0] == 0.0):
        b = b + 1
        y_test.append(0)
    else:
        s = s + 1
        y_test.append(1)
 
y_test = np.array(y_test)
print("testing set: ")
print("0 values " + str(b))
print("1 values " + str(s))

In [ ]:
### 1 is bonafied, 0 is spoofed audio
train_predictions = model.predict(X_train)
confusion = confusion_matrix(y_train, np.argmax(train_predictions,axis=1))
print("training matrix: ")
print(confusion)

In [ ]:
### 1 is bonafied, 0 is spoofed audio
test_predictions = model.predict(X_test)
confusion = confusion_matrix(y_test, np.argmax(test_predictions,axis=1))
print("testing matrix: ")
print(confusion)